## Loading the iris dataset

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
iris = datasets.load_iris()

In [3]:
X = iris.data[:, :4] 
Y = iris.target 
feature_size = len(iris.feature_names)
print(iris.target_names)
target_size = 1
print("Data",X)
print("Target",Y)
print(type(X),type(Y))

['setosa' 'versicolor' 'virginica']
Data [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2

## Splitting the dataset intro train and test

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.20,random_state = 42) 
print(type(X_train),type(y_train))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


## Learning with Context

In [5]:
from torch_mas.head import Head
from torch_mas.data import DataBuffer

model = Head(
    feature_size, 
    target_size, 
    R=0.25,
    imprecise_th=0.5,
    bad_th=0.0015,
    alpha=0.5,
    memory_length=3,
    n_epochs=5
)

## Training the agents

In [6]:
import time
dataset = DataBuffer(X_train,y_train)
t = time.time()
model.fit(dataset)
tt = time.time() - t
print(f"Total training time: {tt}s")

print("Number of agents created:", model.agents.n_agents)

Total training time: 0.5208055973052979s
Number of agents created: 97


## Computing performance

In [7]:
print(type(X_test))

<class 'numpy.ndarray'>


In [8]:
import time
import torch
import math
test_feature_size = X_test.size
t = time.time()
y_pred = model.predict(torch.from_numpy(X_test).float()).round()
y_test = torch.from_numpy(y_test).int()
tt = time.time() - t
tps = tt / test_feature_size
print(f"Total Prediction time: {tt}s for 4 samples")
print(f"Prediction time per sample: {tps}s")

differences = torch.ne(y_pred.squeeze(), y_test)
print(differences)
num_differences = differences.sum().item()
        
print("Differences:", num_differences)

Total Prediction time: 0.0031893253326416016s for 4 samples
Prediction time per sample: 2.657771110534668e-05s
tensor([False,  True,  True, False,  True, False,  True,  True,  True, False,
         True, False, False, False, False,  True,  True, False,  True,  True,
        False, False, False,  True,  True,  True,  True,  True, False, False])
Differences: 16


# Hyperparameter optimisation

In [9]:
dic = {
    "R" : {0,1},
    "bad_th" : {0,1},
    "alpha" : {0,1},
}
init = 0.1
best = math.inf
bestdic = []
for R in range(1,10):
    for b in range(1,10):
        for a in range(1,10):
            model = Head(
            feature_size, 
            target_size, 
            R=R*init,
            imprecise_th=0.5,
            bad_th=b*init,
            alpha=a*init,
            memory_length=3,
            n_epochs=5
            )
            dataset = DataBuffer(X_train,y_train)
            t = time.time()
            model.fit(dataset)
            y_pred = model.predict(torch.from_numpy(X_test).float()).round()
            
            differences = torch.ne(y_pred.squeeze(), y_test)
            num_differences = differences.sum().item()
            #print("Parameters : ",R*init,":",b*init,":",a*init," Score:",num_differences)
            if(num_differences < best):
                bestdic = [{R,b,a}]
                best = num_differences
            elif num_differences == best:
                bestdic.append({R,b,a})
print("Best parameters", bestdic, " with Score", best)

KeyboardInterrupt: 